In [14]:
from google.colab import files
import requests
import os

# === Upload the image ===
print("🔽 Upload an image file for moderation...")
uploaded = files.upload()

if not uploaded:
    print("⚠️ No file uploaded.")
    exit()

image_path = list(uploaded.keys())[0]  # Get the uploaded filename

# === Replace with your actual Sightengine API credentials ===
API_USER = '1338891803'
API_SECRET = 'ssyRXc5HvCe7ELjtaL3nXi3nuoNwKpQL'

# === Image Moderation Function ===
def moderate_image(image_path):
    if not os.path.exists(image_path):
        return {"ok": False, "error": "File not found."}

    with open(image_path, 'rb') as image_file:
        response = requests.post(
            'https://api.sightengine.com/1.0/check.json',
            files={'media': image_file},
            data={
                'models': 'nudity,wad,offensive,gore,text-content',
                'api_user': API_USER,
                'api_secret': API_SECRET
            }
        )

    result = response.json()

    # Show full response (optional for debugging)
    print("\nFull API Response:\n", result)

    if result.get('status') != 'success':
        return {"ok": False, "error": result.get('error', {}).get('message', 'Unknown error')}

    # Check for unsafe content
    if result.get("nudity", {}).get("raw", 0) > 0.5 or result.get("nudity", {}).get("partial", 0) > 0.5:
        return {"ok": False, "reason": "❌ Nudity detected", "nudity": result.get("nudity")}

    if result.get("weapon", 0) > 0.5:
        return {"ok": False, "reason": "❌ Weapon detected", "weapon_score": result["weapon"]}

    if result.get("offensive", {}).get("prob", 0) > 0.5:
        return {"ok": False, "reason": "❌ Offensive content", "offensive_score": result["offensive"]}

    if result.get("gore", {}).get("prob", 0) > 0.5:
        return {"ok": False, "reason": "❌ Gore detected", "gore_score": result["gore"]}

    if result.get("text", {}).get("profanity"):
      return {"ok": False, "reason": "❌ Profane text in image", "text": result["text"]}

    return {"ok": True, "message": "✅ Image is clean and safe to use!"}

# === Run check ===
result = moderate_image(image_path)

print("\n📋 Result:")
for key, value in result.items():
    print(f"{key}: {value}")


🔽 Upload an image file for moderation...


Saving xxx.jpg to xxx.jpg

🔍 Full API Response:
 {'status': 'success', 'request': {'id': 'req_iOHI8yLAzP1xExHOBeFBn', 'timestamp': 1752317654.249602, 'operations': 5}, 'nudity': {'raw': 0.99, 'safe': 0.001, 'partial': 0.001}, 'weapon': 0.001, 'weapon_firearm': 0.001, 'weapon_knife': 0.001, 'alcohol': 0.001, 'drugs': 0.001, 'medical_drugs': 0.001, 'recreational_drugs': 0.001, 'offensive': {'prob': 0.001, 'nazi': 0.001, 'confederate': 0.001, 'supremacist': 0.001, 'terrorist': 0.001, 'middle_finger': 0.001}, 'gore': {'prob': 0.001}, 'skull': {'prob': 0.001}, 'text': {'profanity': [], 'personal': [], 'link': [], 'social': [], 'extremism': [], 'medical': [], 'drug': [], 'weapon': [], 'content-trade': [], 'money-transaction': [], 'spam': [], 'violence': [], 'self-harm': []}, 'media': {'id': 'med_iOHIn92uyxX5LmpjOhoIY', 'uri': 'xxx.jpg'}}

📋 Result:
ok: False
reason: ❌ Nudity detected
nudity: {'raw': 0.99, 'safe': 0.001, 'partial': 0.001}
